In [6]:
from pandas import read_csv
from datetime import datetime
import matplotlib as mpl
from matplotlib import pylab as plt
import matplotlib.dates as mdates
import pandas as pd
import collections
import seaborn as sns
import numpy as np
import tensorflow as tf

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import pickle

np.random.seed(7)

In [7]:
# load data
currencies = ['EXR.D.AUD.EUR.SP00.A', 'EXR.D.BRL.EUR.SP00.A', 'EXR.D.CAD.EUR.SP00.A', 'EXR.D.CNY.EUR.SP00.A', 'EXR.D.GBP.EUR.SP00.A',
              'EXR.D.HKD.EUR.SP00.A', 'EXR.D.IDR.EUR.SP00.A', 'EXR.D.INR.EUR.SP00.A', 'EXR.D.JPY.EUR.SP00.A', 'EXR.D.MYR.EUR.SP00.A']
currencies = ['EXR.D.AUD.EUR.SP00.A', 'EXR.D.BRL.EUR.SP00.A', 'EXR.D.CNY.EUR.SP00.A', 'EXR.D.GBP.EUR.SP00.A',
              'EXR.D.HKD.EUR.SP00.A', 'EXR.D.IDR.EUR.SP00.A', 'EXR.D.INR.EUR.SP00.A', 'EXR.D.JPY.EUR.SP00.A']


currencies = ['EXR.D.INR.EUR.SP00.A']

def builData():
    
    print('Loading data and parsing date')
    
    def parse(x):
        return datetime.strptime(x, '%Y-%m-%d')

    df = read_csv('EURUSD_H1.csv', delimiter='\t')
    print('Data loaded succesfully ')

    # dataset.index.name = 'date'

    # dataset.drop('Unnamed: 0', axis=1, inplace=True)

    # mark all NA values with 0
    keep_cur = ['<CLOSE>']
    
    for curr in df.columns:
        if curr in keep_cur:
            print('Reading currency: ', curr)
        else:
            df.drop(curr, axis=1, inplace=True)  
            print('Deleted currency: ', curr)


    df.info()
    print(df.head(5))
    curr_key = df.columns
    return df

In [8]:
curr_key = ['EURUSD_H1']
#df = builData()
#df.to_pickle('EURUSD_H1')

In [167]:
def prepareData(df):
    #creating train and test sets

    start = 24 * 30*12
    # df.columns.drop(['Unnamed: 0', 'date'])

    dataset = df.values

    dataset = np.array(dataset)
    dataset = dataset[-start:, :].astype(np.float64)
    print(dataset.shape)
    n_curr = len(df.columns)

    train_size = int(dataset.shape[0]*0.8)

    train = dataset[:train_size, :]
    valid = dataset[train_size:, :]
    #converting dataset into x_train and y_train
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(train)
    scaled_data_test = scaler.fit_transform(valid)

    x_train, y_train = [], []
    x_test, y_test = [], []

    lookback = 24
    for i in range(lookback,len(train)):
        x_train.append(scaled_data[i-lookback:i,:])
        y_train.append(0 if scaled_data[i,0] < scaled_data[i-1,0] else 1)
        
    for i in range(lookback,len(valid)):
        x_test.append(scaled_data_test[i-lookback:i,:])
        y_test.append(0 if scaled_data_test[i,0] < scaled_data_test[i-1,0] else 1)

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_test, y_test = np.array(x_test), np.array(y_test)

    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1], n_curr))
    x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1], n_curr))
    
    #indices = np.arange(x_train.shape[0])
    #np.random.shuffle(indices)

    #x_train = x_train[indices, :, :]
    #y_train = y_train[indices, :]
    
    print('x_train 0 ',x_train[0, :, :])
    
    print('y_train 0 ',y_train)
    
    print('x_test 0 ',x_test[0, :, :])
    
    print('y_test 0 ',y_test)

    print(x_train.shape)
    
    print(y_train.shape)
    return x_train, y_train, x_test, y_test, scaler, dataset

In [182]:
from keras.layers.embeddings import Embedding

def buildModel(x_train, y_train, curr_key):
    i = 0
    units = 50
    for curr in curr_key:
        # create and fit the LSTM network
        
        # create the model
        embedding_vecor_length = 24
        model = Sequential()
        model.add(Dense(units, input_dim=24, activation='relu'))
        model.add(Dense(1, activation='softmax'))
        # Compile model
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
               
        X = x_train[:, :, i].reshape(x_train.shape[0], x_train.shape[1])
        y = y_train
        
        model.fit(X, y, epochs=5, batch_size=24)
        # Final evaluation of the model
        scores = model.evaluate(X, y, verbose=0)
        print("Accuracy: %.2f%%" % (scores[1]*100))

        

        filename = curr + '_model_classif.sav'
        pickle.dump(model, open(filename, 'wb'))
        print('Model created :', curr)
        

        i += 1
        model = 0
        
    print('Total created models: ', i)
    

In [183]:
def loadModel(curr_key):
    i = 0
    models = []
    for curr in curr_key:
        # load the model from disk
        filename = curr + '_model_classif.sav'
        loaded_model = pickle.load(open(filename, 'rb'))
        models.append(loaded_model)
        print('Loaded model: ', filename)
    return models

In [184]:
def predict(x_test, models):
    predict_arr = np.zeros((x_test.shape[0], len(models)))

    i = 0
    for model in models:
        X = x_test[:, :, i].reshape(x_test.shape[0], x_test.shape[1])
        
        prediction = model.predict(X)
        predict_arr[:, i] = prediction.reshape(prediction.shape[0],)
        i += 1
    return predict_arr

In [185]:
def getMSE(x_train, y_train, models, scaler):
    i = 0
    mse_arr = []
    ps = 0*y_train
    for model in models:
        X = x_train[:, :, i].reshape(x_train.shape[0], x_train.shape[1], 1)
        p = model.predict(X)
        ps[:, i] = p.reshape(p.shape[0],)
        i += 1

    y_train_inv = scaler.inverse_transform(y_train)
    predict = scaler.inverse_transform(ps)
        
    for j in range(len(models)):
        mse = mean_squared_error(y_train_inv[:, j], predict[:, j])
        mse_arr.append(mse)
    return mse_arr

In [186]:
df = pd.read_pickle('EURUSD_H1')
df.head(5)

,<CLOSE>
0,1.19219
1,1.19220
2,1.19331
3,1.19359
4,1.19199


In [187]:
x_train, y_train, x_test, y_test, scaler, curr_values = prepareData(df)

(8640, 1)
x_train 0  [[0.69134727]
 [0.69918699]
 [0.70441347]
 [0.70136469]
 [0.69381533]
 [0.69091173]
 [0.69976771]
 [0.69686411]
 [0.70078397]
 [0.69555749]
 [0.69657375]
 [0.69831591]
 [0.69642857]
 [0.67958769]
 [0.68873403]
 [0.59712544]
 [0.5988676 ]
 [0.6054007 ]
 [0.58623693]
 [0.59204413]
 [0.57026713]
 [0.58478513]
 [0.60438444]
 [0.60728804]]
y_train 0  [1 1 0 ... 0 0 0]
x_test 0  [[0.4417208 ]
 [0.44159824]
 [0.43019978]
 [0.42615517]
 [0.42615517]
 [0.42517465]
 [0.42529722]
 [0.42063978]
 [0.41426645]
 [0.41843363]
 [0.42223312]
 [0.42811619]
 [0.4294644 ]
 [0.42529722]
 [0.44846182]
 [0.43167055]
 [0.43571516]
 [0.43571516]
 [0.44049516]
 [0.43939208]
 [0.43706337]
 [0.43841157]
 [0.44196593]
 [0.44441721]]
y_test 0  [0 0 1 ... 0 0 1]
(6888, 24, 1)
(6888,)


In [188]:
print(curr_key)
buildModel(x_train, y_train, curr_key)

['EURUSD_H1']
Epoch 1/5
6888/6888 [==============================] - 1s 82us/step - loss: 7.5509 - accuracy: 0.5075
Epoch 2/5
6888/6888 [==============================] - 0s 72us/step - loss: 7.5509 - accuracy: 0.5075
Epoch 3/5
6888/6888 [==============================] - 0s 68us/step - loss: 7.5509 - accuracy: 0.5075
Epoch 4/5
6888/6888 [==============================] - 0s 67us/step - loss: 7.5509 - accuracy: 0.5075
Epoch 5/5
6888/6888 [==============================] - 0s 68us/step - loss: 7.5509 - accuracy: 0.5075
Accuracy: 50.75%
Model created : EURUSD_H1
Total created models:  1


In [161]:
print(curr_key)

['EURUSD_H1']


In [162]:
models = loadModel(curr_key)

Loaded model:  EURUSD_H1_model_classif.sav


In [163]:
predict_arr = predict(x_test, models)
predict_arr = [0 if predict_arr[i] < 0.5 else 1 for i in range(len(predict_arr))]

In [164]:
#y_test_inv = scaler.inverse_transform(y_test)
#predict = scaler.inverse_transform(predict_arr)
horizon = 24*10

In [165]:
def getActions(y, V, Cash, T):
    n = 1
    t_cur = T
    t_prev = t_cur - 1
    t_prev2 = t_prev - 1
    t_next = t_cur + 1
    
    X_eur = Cash[t_cur, -1]
    X_new = X_eur
    A = []
    
    for c in range(n):
        action = ''
        X = 0

        d_cur = y[t_cur] - y[t_prev]
        d_next = V[T]

        if d_cur > 0 and d_next == 0:
            action = 'BUY'

        elif d_cur < 0 and d_next == 1:
            action = 'SELL'
        A.append([action, X])
    
    return A

In [166]:
V = predict_arr[-horizon:]
y = y_test[-horizon:]
cur_val = curr_values[-horizon:, 0]
print(len(cur_val))
n = 1
initial_cash = 1000
X_buy = 0*V
X_sell = 0*V

Cash = np.zeros((horizon, n + 1))
Cash[0, -1] = initial_cash
print('n ', n)

for t in range(horizon):
    A = getActions(y, V, Cash, t)
    if t >0: Cash[t, :] = Cash[t-1, :]
    b = 0
    c = 0
    
    print('Heure ', t)
    for a in A:        
        if a[0] == 'SELL' and Cash[t, c] > 0:
            print(str(a[0]) + ' ' + str(Cash[t, c]) + ' ' + str(curr_key[c]) + ' rate :' + str(cur_val[t]))
            
            Cash[t, -1] = Cash[t, -1] + Cash[t, c]/cur_val[t]
            Cash[t, c] = Cash[t, c] -  Cash[t, c]
            
            
            print('Euro after sell ', Cash[t, -1])
        if a[0] == 'BUY':
            b += 1
        c += 1
          
    c = 0    
    for a in A:
            
        if a[0] == 'BUY' and Cash[t, -1] > 0:
            
            print(str(a[0]) + ' ' + str(Cash[t, -1]*cur_val[t]/b) + ' ' + str(curr_key[c]) + ' rate: ' + str(cur_val[t]/b))
            
            Cash[t, c] = Cash[t, c] + Cash[t, -1]*cur_val[t]/b
            Cash[t, -1] = Cash[t, -1] - Cash[t, -1]/b
            
            print('Euro after buy ', Cash[t, -1])
        
        c += 1
    
    print('Actions end of day: ', A)
    print('Accounts status: ', Cash[t, :])
    last_day = t

final_cash = Cash[last_day, -1]
print(final_cash)
for i in range(n):
    if y[last_day] != 0:
        final_cash += Cash[last_day, i]/cur_val[last_day]
        initial_cash += Cash[0, i]/cur_val[0]

d_growth = (final_cash/initial_cash)**(1/(horizon/24)) - 1
                 
print('Initial cash value (eur): ', initial_cash)
print('Final cash value (eur): ', final_cash)
print('Daily growth: ', d_growth, '%')
print('Win: ', final_cash - initial_cash)

240
n  1
Heure  0
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  1
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  2
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  3
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  4
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  5
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  6
Actions end of day:  [['SELL', 0]]
Accounts status:  [   0. 1000.]
Heure  7
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  8
Actions end of day:  [['SELL', 0]]
Accounts status:  [   0. 1000.]
Heure  9
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  10
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  11
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  12
Actions end of day:  [['', 0]]
Accounts status:  [   0. 1000.]
Heure  13
Actions end of day:  [['', 0]]
Acc

In [95]:
mse_arr = getMSE(x_train, y_train, models, scaler)
for i in range(len(curr_key)):
    xs = np.arange(int(horizon))
    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 1, 1)

    ax.plot(xs, y_test_inv[-horizon:, i], label='Real ' + curr_key[i], lw=2, color='black')
    ax.plot(xs, predict[-horizon:, i], label='Forecast ' + curr_key[i], lw=2, color='blue')
    

    rmse = mse_arr[i]
    print('rmse: ', rmse)

    rmse_max = [float(p) + float(rmse)/2 for p in predict[:, i]]
    rmse_min = [float(p) - float(rmse)/2 for p in predict[:, i]]

    ax.fill_between(xs, rmse_max[-horizon:], rmse_min[-horizon:], color='red', alpha=0.2)
    plt.show()

IndexError: too many indices for array

In [ ]:
-